# Altegrad Experiments

In [21]:
import pandas as pd
import os

def from_txt_to_csv(file_name, folder_name):
    """
    Transform the output of the read_results_predict.py in a proper Kaggle Submission, i.e : a well formated csv file
    
    inputs : 
    - file_name is the name (string) of the txt file generated by the read_results_predict.py (without the .txt extension)
    - folder_name is the desired or existing name (string) of the folder where the submission will be stored into 
    
    output : None
    
    """
    # change me !
    path_root = os.path.dirname(os.path.abspath('').replace('\\', '/')) + "/data/"
    path_file = path_root  + file_name + ".txt"
    path_folder = path_root + folder_name
    # create a directory if it doesn't exist yet
    if not os.path.exists(path_folder):
        os.makedirs(path_folder)
      
    df = pd.read_csv(path_file)
    df.to_csv(path_folder + "/" + file_name + ".csv", index=False)
    print("Submission saved in '{}'".format(path_folder))
    

In [24]:
from_txt_to_csv('predictions_han_tgt2', '')

Submission saved in 'D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/HAN_Graphs/data/'


In [2]:
!python ./experiments/target_0/attention_mech/attention_training.py


Epoch 00001: val_loss improved from inf to 0.36101, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/HAN_Graphs/data/model_0

Epoch 00002: val_loss improved from 0.36101 to 0.29493, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/HAN_Graphs/data/model_0

Epoch 00003: val_loss improved from 0.29493 to 0.26520, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/HAN_Graphs/data/model_0

Epoch 00004: val_loss improved from 0.26520 to 0.26241, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/HAN_Graphs/data/model_0

Epoch 00005: val_loss improved from 0.26241 to 0.25303, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/HAN_Graphs/data/model_0

Epoch 00006: val_loss improved from 0.25303 to 0.23532, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/HAN_Graphs/data/model_0

Epoch 00007: val_loss did not improve from 0.2353

Using TensorFlow backend.


In [10]:
!python ./experiments/adding_layers/read_results_predict.py

Using TensorFlow backend.


# CODE TO TRAIN ONE TARGET

In [14]:
import sys
import json
import numpy as np

from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input, Embedding, Dropout, Bidirectional, GRU, CuDNNGRU, TimeDistributed, Dense

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

tgt = 2

is_GPU = True
save_hist = False
save_weights = True
path_root = os.path.dirname(os.path.abspath('').replace('\\', '/'))
path_to_data = path_root + '/data/'
path_to_code = path_root + '/code/experiments/target_' + str(tgt) + '/'
sys.path.insert(0, path_to_code)

# = = = = = = = = = = = = = = =

from AttentionWithContext import AttentionWithContext
from make_model_tgt2 import make_model

# = = = = = hyper-parameters = = = = =


n_units = 80
drop_rate = 0.2
batch_size = 96
nb_epochs = 60
my_optimizer = 'adam'
my_patience = 6

# = = = = = data loading = = = = =

docs = np.load(path_to_data + 'documents.npy')
embeddings = np.load(path_to_data + 'embeddings.npy')

with open(path_to_data + 'train_idxs.txt', 'r') as file:
    train_idxs = file.read().splitlines()
    
train_idxs = [int(elt) for elt in train_idxs]

idxs_select_train = np.random.choice(range(len(train_idxs)), size=int(len(train_idxs) * 0.80), replace=False)
idxs_select_val = np.setdiff1d(range(len(train_idxs)), idxs_select_train)

train_idxs_new = [train_idxs[elt] for elt in idxs_select_train]
val_idxs = [train_idxs[elt] for elt in idxs_select_val]

docs_train = docs[train_idxs_new, :, :]
docs_val = docs[val_idxs, :, :]


with open(path_to_data + 'targets/train/target_' + str(tgt) + '.txt', 'r') as file:
    target = file.read().splitlines()
    
target_train = np.array([target[elt] for elt in idxs_select_train]).astype('float')
target_val = np.array([target[elt] for elt in idxs_select_val]).astype('float')

model = make_model(n_units, drop_rate, embeddings, docs_train, is_GPU)
print(model.summary())
 
model.compile(loss='mean_squared_error',
                  optimizer=my_optimizer,
                  metrics=['mse'])

# = = = = = training = = = = =

early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=my_patience,
                                   mode='min')

# save model corresponding to best epoch
checkpointer = ModelCheckpoint(filepath=path_to_data + 'model_' + str(tgt), 
                                   verbose=1, 
                                   save_best_only=True,
                                   save_weights_only=True)

if save_weights:
    my_callbacks = [early_stopping, checkpointer]
else:
    my_callbacks = [early_stopping]

model.fit(docs_train, 
              target_train,
              batch_size = batch_size,
              epochs = nb_epochs,
              validation_data = (docs_val, target_val),
              callbacks = my_callbacks)
    

if save_hist:
    hist = model.history.history
    with open(path_to_data + 'model_history_' + str(tgt) + '.json', 'w') as file:
        json.dump(hist, file, sort_keys=False, indent=4)

print('* * * * * * * target', tgt, 'done * * * * * * *')    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 70, 11)            0         
_________________________________________________________________
time_distributed_16 (TimeDis (None, 70, 80)            22013995  
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 70, 80)            77760     
_________________________________________________________________
attention_with_context_16 (A (None, 80)                6560      
_________________________________________________________________
dropout_16 (Dropout)         (None, 80)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 80)                6480      
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 81        
Total para

59980/59980 [==============================] - 42s 708us/step - loss: 0.3332 - mean_squared_error: 0.3332 - val_loss: 0.3929 - val_mean_squared_error: 0.3929

Epoch 00025: val_loss did not improve from 0.38863
Epoch 26/60
59980/59980 [==============================] - 42s 706us/step - loss: 0.3143 - mean_squared_error: 0.3143 - val_loss: 0.3825 - val_mean_squared_error: 0.3825

Epoch 00026: val_loss improved from 0.38863 to 0.38254, saving model to D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/HAN_Graphs/data/model_2
Epoch 27/60
59980/59980 [==============================] - 42s 702us/step - loss: 0.3042 - mean_squared_error: 0.3042 - val_loss: 0.3904 - val_mean_squared_error: 0.3904

Epoch 00027: val_loss did not improve from 0.38254
Epoch 28/60
59980/59980 [==============================] - 42s 708us/step - loss: 0.3079 - mean_squared_error: 0.3079 - val_loss: 0.3833 - val_mean_squared_error: 0.3833

Epoch 00028: val_loss did not improve from 0.38254
Epoch 29/60
59980

# CODE TO PREDICT ONE TARGET

In [52]:
import sys
import json
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Input, Embedding, Dropout, Bidirectional, GRU, CuDNNGRU, TimeDistributed, Dense

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# = = = = = = = = = = = = = = =

tgt = 2

is_GPU = True
save_hist = False
path_root = os.path.dirname(os.path.abspath('').replace('\\', '/'))
path_to_data = path_root + '/data/'
path_to_code = path_root + '/code/experiments/target_' + str(tgt) + '/'
sys.path.insert(0, path_to_code)

# = = = = = = = = = = = = = = =

from AttentionWithContext import AttentionWithContext
from make_model_tgt2 import make_model
# = = = = = = = = = = = = = = =

docs = np.load(path_to_data + 'documents.npy')
embeddings = np.load(path_to_data + 'embeddings.npy')

with open(path_to_data + 'test_idxs.txt', 'r') as file:
    test_idxs = file.read().splitlines()


test_idxs = [int(elt) for elt in test_idxs]
docs_test = docs[test_idxs,:,:]

all_preds_han = []



indx_tg0 = [i for i in range(0, 18744)]
indx_tg1 = [i for i in range(18744, 37488)]
indx_tg2 = [i for i in range(37488, 56232)]
indx_tg3 = [i for i in range(56232,74976)]
    
# * * * HAN * * * 
    
# relevant hyper-parameters
n_units = 80
drop_rate = 0 # prediction mode
 
model = make_model(n_units, drop_rate, embeddings, docs_test, is_GPU)
    
model.load_weights(path_to_data + 'model_' + str(tgt))
all_preds_han.append(model.predict(docs_test).tolist())

# flatten
all_preds_han = [elt[0] for sublist in all_preds_han for elt in sublist]

# write the predictions of a single target with the corrects indx
with open(path_to_data + 'predictions_han_' + str(tgt) + '.txt', 'w') as file:
    for idx, pred in zip(indx_tg2, all_preds_han):
        pred = format(pred, '.7f')
        file.write(str(idx) + ',' + pred + '\n')
        


In [65]:
import pandas as pd 

preds = pd.read_csv(path_to_data + 'predictions_han.csv')
from_txt_to_csv('predictions_han_2', '')

Submission saved in 'D:/Scolaire/Code/Python/Machine_Learning/Kaggle_Challenges_M2/HAN_Graphs/data/'
